In [123]:
import pandas as pd
import numpy as np
data=pd.read_csv('train.csv')
data.head(2)

,ID,Place,location,date,status,job_title,summary,positives,negatives,advice_to_mgmt,score_1,score_2,score_3,score_4,score_5,score_6,overall
0,1,startup_1,NaN,"Dec 11, 2018",Current Employee,Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4.0,5.0,5.0,4.0,5.0,0,5.0
1,2,startup_1,"Mountain View, CA","Jun 21, 2013",Former Employee,Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2.0,3.0,3.0,5.0,3.0,2094,5.0


In [124]:
#### Analysing the data
print (data.Place.isnull().sum())          # no null
print (data.location.isnull().sum())       # 11254 null entry
print (data.date.isnull().sum())           # no null
print (data.status.isnull().sum())         # no null
print (data.job_title.isnull().sum())      # no null
print (data.summary.isnull().sum())        # 52 null entry
print (data.positives.isnull().sum())      # no null
print (data.negatives.isnull().sum())      # no null
print (data.advice_to_mgmt.isnull().sum()) # 13277 null entry  #advice_to_mgmt has more number of null entry we can ditch it
print (data.score_1.isnull().sum())        # 3186 null entry
print (data.score_2.isnull().sum())        # 6050 null entry
print (data.score_3.isnull().sum())        # 3169 null entry
print (data.score_4.isnull().sum())        # 3191 null entry
print (data.score_5.isnull().sum())        # 3485 null entry
print (data.score_6.isnull().sum())        # no null
print (data.overall.isnull().sum())        # no null

0
11254
0
0
0
52
0
0
13277
3186
6050
3169
3191
3485
0
0


In [125]:
##Droping data that seems to have no contribution for rating prediction(keeping only reviews provided)
data=data.drop(['ID','Place','location','date','status','job_title','advice_to_mgmt'],axis=1)
#data=data.drop(['ID','Place','summary','location','date','status','job_title','advice_to_mgmt'],axis=1)
data.head(2)

,summary,positives,negatives,score_1,score_2,score_3,score_4,score_5,score_6,overall
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,4.0,5.0,5.0,4.0,5.0,0,5.0
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,2.0,3.0,3.0,5.0,3.0,2094,5.0


In [126]:
##checking the most frequent data entry in out features  
#print(data["summary"].value_counts())         ##"Great company" has occued 224 times
#print(data["advice_to_mgmt"].value_counts())  ##advice_to_mgmt has more number of null entry we can ditch it
print(data["score_1"].value_counts())
print(data["score_2"].value_counts())
print(data["score_3"].value_counts())
print(data["score_4"].value_counts())
print(data["score_5"].value_counts())
print(data["score_6"].value_counts())

4.0    6830
5.0    6362
3.0    6238
2.0    3540
1.0    3220
3.5     352
4.5     339
2.5     203
1.5      66
Name: score_1, dtype: int64
5.0    9736
4.0    6103
3.0    4154
1.0    2223
2.0    2070
Name: score_2, dtype: int64
5.0    8089
4.0    7569
3.0    5986
2.0    2749
1.0    2063
3.5     298
4.5     183
2.5     166
1.5      64
Name: score_3, dtype: int64
5.0    9609
4.0    8919
3.0    5089
2.0    1546
1.0     887
4.5     473
3.5     435
2.5     161
1.5      26
Name: score_4, dtype: int64
4.0    7039
3.0    6173
5.0    6018
2.0    3576
1.0    3481
3.5     192
2.5     148
4.5     145
1.5      79
Name: score_5, dtype: int64
0       20206
1        4741
2        2040
3        1066
4         603
5         401
6         268
7         183
8         161
9         116
10         93
11         64
12         46
14         40
13         38
15         36
16         24
18         23
23         17
17         16
19         16
22         13
20         13
24         13
27          9
25          7
21  

In [128]:
data['summary']=data['summary'].fillna('Great company')
data['score_1']=data['score_1'].fillna(3.0)
data['score_2']=data['score_2'].fillna(3.0)
data['score_3']=data['score_3'].fillna(3.0)
data['score_4']=data['score_4'].fillna(3.0)
data['score_5']=data['score_5'].fillna(3.0)
data['score_6']=data['score_6'].fillna(3.0)

In [129]:
data.head(2)

,summary,positives,negatives,score_1,score_2,score_3,score_4,score_5,score_6,overall
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,4.0,5.0,5.0,4.0,5.0,0,5.0
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,2.0,3.0,3.0,5.0,3.0,2094,5.0


In [130]:
# Feature Engineering
from nltk.corpus import stopwords
sw = stopwords.words("english")
import re
from nltk.stem.snowball import SnowballStemmer,PorterStemmer
import string
def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower().split()
    ## Remove stop words
    text = [w for w in text if not w in sw and len(w) >= 3]
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    txt = text.split()
    text = [i for i in txt if i.isalpha()]
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    text=text.lower()
    return text
#summary=data['summary']
#summary=summary.apply(clean_text)
data['summary']=data['summary'].apply(clean_text)
data['positives']=data['positives'].apply(clean_text)
data['negatives']=data['negatives'].apply(clean_text)
data.head(2)

,summary,positives,negatives,score_1,score_2,score_3,score_4,score_5,score_6,overall
0,best compani work,peopl smart friend,bureaucraci slow thing,4.0,5.0,5.0,4.0,5.0,0,5.0
1,move speed light burn inevit,food food food cafe main campus mtv alon mini ...,work life balanc balanc perk benefit illus kee...,2.0,3.0,3.0,5.0,3.0,2094,5.0


In [139]:
tf_train_data = pd.concat([data['summary'],data['positives'],data['negatives']])
#tf_train_data = pd.concat([data['positives'],data['negatives']])
tf_train_data.head()

0                                    best compani work
1                         move speed light burn inevit
2    great balanc big compani secur fun fast move p...
3                   best place i have work also demand
4                                         execel engin
dtype: object

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df=3, max_features=1000,strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)
tfv.fit(tf_train_data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents='unicode', sublinear_tf=1,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=1,
        vocabulary=None)

In [134]:
desc1 = tfv.transform(list(data["summary"]))
desc2 = tfv.transform(list(data["positives"]))
desc3 = tfv.transform(list(data["negatives"]))

In [135]:
print (len(desc1.toarray()))
print (len(desc2.toarray()))
print (len(desc3.toarray()))

30336
30336
30336


In [137]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=40)
desc11 = svd.fit_transform(desc1)
desc21 = svd.fit_transform(desc2)
desc31 = svd.fit_transform(desc3)

desc11 = pd.DataFrame(desc11,columns=['nlp_1_{}'.format(i) for i in range(40)])
desc21 = pd.DataFrame(desc21,columns=['nlp_1_{}'.format(i) for i in range(40)])
desc31 = pd.DataFrame(desc31,columns=['nlp_1_{}'.format(i) for i in range(40)])

#desc11 = pd.DataFrame(desc11)
#desc21 = pd.DataFrame(desc21)
#desc31 = pd.DataFrame(desc31)


final_df = pd.concat([data,desc11,desc21,desc31],axis=1, sort=False)
#final_df = pd.concat([data,desc21,desc31],axis=1, sort=False)

In [140]:
#desc11 = pd.DataFrame(desc11.toarray(), columns=test_tfvec.get_feature_names())
final_df.head(2)

,summary,positives,negatives,score_1,score_2,score_3,score_4,score_5,score_6,overall,...,nlp_1_30,nlp_1_31,nlp_1_32,nlp_1_33,nlp_1_34,nlp_1_35,nlp_1_36,nlp_1_37,nlp_1_38,nlp_1_39
0,best compani work,peopl smart friend,bureaucraci slow thing,4.0,5.0,5.0,4.0,5.0,0,5.0,...,0.125454,-0.112908,0.003553,0.017763,0.087155,-0.007532,0.26091,0.012537,-0.156342,0.186754
1,move speed light burn inevit,food food food cafe main campus mtv alon mini ...,work life balanc balanc perk benefit illus kee...,2.0,3.0,3.0,5.0,3.0,2094,5.0,...,0.019885,0.024984,0.002274,0.064493,-0.018545,-0.010730,-0.08054,0.000355,-0.053892,0.042879


In [141]:
Y = final_df["overall"]
final_df = final_df.drop(["summary","positives", "negatives", "overall"], axis =1)
#final_df = final_df.drop(["positives", "negatives", "overall"], axis =1)
final_df.head(1)

,score_1,score_2,score_3,score_4,score_5,score_6,nlp_1_0,nlp_1_1,nlp_1_2,nlp_1_3,...,nlp_1_30,nlp_1_31,nlp_1_32,nlp_1_33,nlp_1_34,nlp_1_35,nlp_1_36,nlp_1_37,nlp_1_38,nlp_1_39
0,4.0,5.0,5.0,4.0,5.0,0,0.187555,0.17126,0.097596,0.000501,...,0.125454,-0.112908,0.003553,0.017763,0.087155,-0.007532,0.26091,0.012537,-0.156342,0.186754


In [142]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(final_df, Y, test_size=0.25)

In [143]:
##Logistic Regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', C=1,multi_class='multinomial').fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
from sklearn.metrics import accuracy_score
acc_lr = accuracy_score(Y_test, Y_pred)
acc_lr

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.39609704641350213

In [144]:
Y_pred

array([3., 4., 3., ..., 3., 4., 2.])

In [145]:
##Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0).fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc_dt = accuracy_score(Y_test, Y_pred)
acc_dt

0.3367616033755274

In [146]:
Y_pred

array([2., 3., 3., ..., 4., 4., 3.])

In [147]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=400,max_depth=2)
rfc.fit(X_train,Y_train)
Y_pred = rfc.predict(X_test)
acc_dt = accuracy_score(Y_test, Y_pred)
acc_dt

0.38739451476793246

In [148]:
##SVC 
from sklearn.svm import SVC
sv=SVC()
sv.fit(X_train,Y_train)
y_pred=sv.predict(X_test)
accuracy_score(Y_test, Y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.38739451476793246

In [116]:
'''
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train,Y_train)
Y_pred = xgb.predict(X_test)
acc_dt = accuracy_score(Y_test, Y_pred)
acc_dt

acc=[]
lr = [0.08, 0.2, 0.4]
estimator =[ 300,400, 500]
for a in lr:
    for e in estimator:
        xgb = XGBClassifier(learning_rate=a,n_estimators=e, booster='dart')
        xgb.fit(X_train, Y_train)
        Y_pred_xgb = xgb.predict(X_test)
        Y_pred_xgb = Y_pred_xgb.tolist()
        accr = accuracy_score(Y_test, Y_pred_xgb)
        print("Acc for lr & estimator", a,e, accr)
        acc.append(accr)  
'''

'\nfrom xgboost import XGBClassifier\nxgb=XGBClassifier()\nxgb.fit(X_train,Y_train)\nY_pred = xgb.predict(X_test)\nacc_dt = accuracy_score(Y_test, Y_pred)\nacc_dt\n\nacc=[]\nlr = [0.08, 0.2, 0.4]\nestimator =[ 300,400, 500]\nfor a in lr:\n    for e in estimator:\n        xgb = XGBClassifier(learning_rate=a,n_estimators=e, booster=\'dart\')\n        xgb.fit(X_train, Y_train)\n        Y_pred_xgb = xgb.predict(X_test)\n        Y_pred_xgb = Y_pred_xgb.tolist()\n        accr = accuracy_score(Y_test, Y_pred_xgb)\n        print("Acc for lr & estimator", a,e, accr)\n        acc.append(accr)  \n'

In [151]:
from sklearn.linear_model import SGDClassifier
sgd=SGDClassifier()
sgd.fit(X_train,Y_train)
y_pred=sv.predict(X_test)
accuracy_score(Y_test, Y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.38739451476793246

In [150]:
from sklearn.linear_model import RidgeClassifier
rc=SGDClassifier()
rc.fit(X_train,Y_train)
y_pred=rc.predict(X_test)
accuracy_score(Y_test, Y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.38739451476793246

In [149]:
'''
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=200,learning_rate=1,loss_function='MultiClass',depth = 2)# loss_function='CrossEntropy')
model.fit(X_train, Y_train,eval_set=(X_test, Y_test),verbose=False)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())
'''

"\nfrom catboost import CatBoostClassifier\nmodel = CatBoostClassifier(iterations=200,learning_rate=1,loss_function='MultiClass',depth = 2)# loss_function='CrossEntropy')\nmodel.fit(X_train, Y_train,eval_set=(X_test, Y_test),verbose=False)\nprint('Model is fitted: ' + str(model.is_fitted()))\nprint('Model params:')\nprint(model.get_params())\n"

In [153]:
test=pd.read_csv('test.csv')
test=test.drop(['ID','Place','location','date','status','job_title','advice_to_mgmt'],axis=1)
test.head(2)

,summary,positives,negatives,score_1,score_2,score_3,score_4,score_5,score_6
0,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",5,5,5,5,5,49
1,NICE working in GOOGLE as an INTERN,"People are not that busy, so they are nice to ...",Food is not good as I expected. People said it...,4,4,4,5,4,1


In [155]:
test['summary']=test['summary'].fillna('Great company')
test['score_1']=test['score_1'].fillna(3.0)
test['score_2']=test['score_2'].fillna(3.0)
test['score_3']=test['score_3'].fillna(3.0)
test['score_4']=test['score_4'].fillna(3.0)
test['score_5']=test['score_5'].fillna(3.0)
test['score_6']=test['score_6'].fillna(3.0)

In [157]:
print (test.summary.isnull().sum())        # 0 null entry
print (test.positives.isnull().sum())      # 0 null
print (test.negatives.isnull().sum())      # 1 null
print (test.score_1.isnull().sum())        # 0 null entry
print (test.score_2.isnull().sum())        # 0 null entry
print (test.score_3.isnull().sum())        # 0 null entry
print (test.score_4.isnull().sum())        # 0 null entry
print (test.score_5.isnull().sum())        # 0 null entry
print (test.score_6.isnull().sum())        # no null

0
0
1
0
0
0
0
0
0


In [164]:
#test.negatives.value_counts()
test['negatives']=test['negatives'].fillna('None that I can think of.')  ##Most common

In [165]:
test["summary"] = test["summary"].apply(clean_text)
test["positives"] = test["positives"].apply(clean_text)
test["negatives"] = test["negatives"].apply(clean_text)

In [174]:
desc1 = tfv.transform(list(test["summary"]))
desc2 = tfv.transform(list(test["positives"]))
desc3 = tfv.transform(list(test["negatives"]))

desc1 = svd.fit_transform(desc1)
desc2 = svd.fit_transform(desc2)
desc3 = svd.fit_transform(desc3)

desc1 = pd.DataFrame(desc1, columns=['nlp_1_{}'.format(i) for i in range(40)])
desc2 = pd.DataFrame(desc2, columns=['nlp_2_{}'.format(i) for i in range(40)])
desc3 = pd.DataFrame(desc3, columns=['nlp_3_{}'.format(i) for i in range(40)])

final_df_test = pd.concat([test,desc1,desc2,desc3],axis=1, sort=False)

In [175]:
final_df_test.head(1)

,summary,positives,negatives,score_1,score_2,score_3,score_4,score_5,score_6,nlp_1_0,...,nlp_3_30,nlp_3_31,nlp_3_32,nlp_3_33,nlp_3_34,nlp_3_35,nlp_3_36,nlp_3_37,nlp_3_38,nlp_3_39
0,uniqu one kind dream job,googl world own everi compani lot peopl seriou...,work mtv hq given work howev passion want adva...,5,5,5,5,5,49,0.013881,...,0.005003,0.023659,0.028894,-0.027254,0.067287,0.049028,-0.062256,-0.033575,0.055727,0.013482


In [176]:
final_df_test = final_df_test.drop(["summary", "positives", "negatives"], axis=1)
final_df_test.shape

(29272, 126)

In [177]:
test_pred= clf.predict(final_df_test)

In [179]:
test_df=pd.read_csv('	rain.csv')

In [180]:
test_pred = test_pred.tolist()
final_submission=pd.DataFrame({'overall':test_pred})
final_submission.insert(loc=0,column='ID',value=test_df["ID"])

In [181]:
final_submission.to_csv('submission_final.csv',index=False)